<a href="https://colab.research.google.com/github/Hamayah/BT3104_Group_Project/blob/main/BT3104_Group_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brief Description
"Default of Credit Card Clients Dataset"

It contains information about the payment behavior of 30,000 credit card holders from a bank in Taiwan. There are 23 feature attributes (B to X), such as credit limit, payment history, and demographic details, that are used to predict whether a client will default on their next payment. The target variable (column Y) is binary: 0 for "not default" and 1 for "default".

The data modeling problem is to predict the likelihood of credit card default based on these 23 features. The goal is to use machine learning models to analyze which factors are significant predictors and to build a model that accurately classifies clients as either "default" or "not default".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/bt3104/

 card.csv  'Group 1.ipynb'   Untitled0.ipynb


In [ ]:
import pandas as pd
import numpy as np

# Read the data, skipping the first two lines for headers
data = pd.read_csv("/content/drive/MyDrive/bt3104/card.csv", sep=",", skiprows=2, header=None)

# Extracting the header (first two lines)
with open("/content/drive/MyDrive/bt3104/card.csv", "r") as f:
    header = [next(f).strip().split(",") for _ in range(2)]

# Setting a random seed for reproducibility
np.random.seed(1234)

# Splitting the data into training and testing sets
n = len(data)
index = np.arange(n)
test_index = np.random.choice(index, size=int(n/4), replace=False)
train_index = np.setdiff1d(index, test_index)

test_data = data.iloc[test_index]
train_data = data.iloc[train_index]

print("Test Data: \n\n", test_data)

Test Data: 

           0       1   2   3   4   5   6   7   8   9   ...     15     16  \
13125  13126  400000   1   1   1  34  -2  -2  -2  -2  ...      0      0   
14635  14636   80000   1   2   2  34   0   0   0   0  ...  43182  44332   
19429  19430  200000   2   3   1  49   1  -2  -1  -1  ...   7588   7606   
4381    4382   20000   2   2   1  41  -1  -1  -1  -1  ...      0   7014   
7659    7660   70000   2   1   1  36   2   0   0   0  ...  65287  35345   
...      ...     ...  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...   
11494  11495  500000   2   1   2  38   0   0   0   0  ...   2043   8479   
25558  25559  110000   2   1   2  28   0   0   0   0  ...  84362  59391   
17521  17522  150000   2   6   1  47   0   0   0   0  ...  93063  92758   
13310  13311  140000   1   2   1  42   0   0   0   0  ...  65589  69117   
2674    2675   80000   2   3   1  40   0   0   0   0  ...  25722  26261   

          17    18     19    20    21     22     23  24  
13125      0     0      0  

#